# Application of the flow__distance utility on a Sicilian basin

This notebook illustrates how to run the ``flow__distance`` utility on a digital elevation model (DEM) that represents a real basin in Sicily. First, a watershed will be extracted from the input DEM by using the watershed utility. Then, the distances from each node to the watershed's outlet will be obtained with the ``flow__distance`` utility. Flow is routed using the D8 algorithm.

First, import what we'll need:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from landlab.components import FlowAccumulator
from landlab.io import esri_ascii
from landlab.utils import watershed
from landlab.utils.flow__distance import calculate_flow__distance

Import a square DEM that includes the watershed:

In [ ]:
with open("nocella_resampled.txt") as fp:
    mg = esri_ascii.load(fp, name="topographic__elevation", at="node")

Run the FlowAccumulator and the DepressionFinderAndRouter components to find depressions, to route the flow across them and to calculate flow direction and drainage area:

In [ ]:
fr = FlowAccumulator(
    mg, flow_director="D8", depression_finder="DepressionFinderAndRouter"
)
fr.run_one_step()

Set the id of the outlet. The value indicated here is the node id of the entire watershed's outlet:

In [ ]:
outlet_node = 15324

Run the watershed utility and show the watershed mask:

In [ ]:
ws_mask = watershed.get_watershed_mask(mg, outlet_node)
mg.imshow(ws_mask, allow_colorbar=False)
plt.plot(mg.x_of_node[outlet_node], mg.y_of_node[outlet_node], "*", markersize=20)

Run the ``flow__distance`` utility:

In [ ]:
flow_distance = calculate_flow__distance(mg, add_to_grid=True, clobber=True)

Mask the ``flow__distance`` to the watershed mask. This operation has to be done because the ``flow__distance`` utility is applied to the entire grid that contains other streams not connected with our stream network and, for this reason, not belonging to our watershed.

In [ ]:
flow_distance_to_outlet = np.zeros(mg.number_of_nodes)
flow_distance_to_outlet[ws_mask] = flow_distance[ws_mask] - flow_distance[outlet_node]

Plot the spatial distribution of the distances from each node to the watershed's outlet:

In [ ]:
mg.imshow(flow_distance_to_outlet, colorbar_label="flow distance (m)")
plt.plot(mg.x_of_node[outlet_node], mg.y_of_node[outlet_node], "*", markersize=20)